In [ ]:
#https://www.dataquest.io/blog/introduction-to-ensembles/
#http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
#https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html

In [16]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline
from mlens.ensemble import SuperLearner

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

##from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.svm import SVC
#from catboost import CatBoostClassifier
#from sklearn.metrics import log_loss

In [17]:
datadir="D:/Machine Learning/Poverty_comp/data/"
a_train=pd.read_csv(datadir+'A_hhold_train.csv', index_col='id')
b_train=pd.read_csv(datadir+'A_hhold_train.csv', index_col='id')
c_train=pd.read_csv(datadir+'A_hhold_train.csv', index_col='id')

In [18]:
b_train.poor.value_counts(normalize=False).plot.bar(title='Number of Poor for country A')

In [19]:
# Standardize features
def standardize(df, numeric_only=True):
    numeric = df.select_dtypes(include=['int64', 'float64'])
    
    # subtracy mean and divide by std
    #df[numeric.columns] = (numeric - numeric.mean()) / numeric.std()
    
    return df
    

def pre_process_data(df, enforce_cols=None):
    print("Input shape:\t{}".format(df.shape))
        

    df = standardize(df)
    print("After standardization {}".format(df.shape))
        
    # create dummy variables for categoricals
    df = pd.get_dummies(df)
    print("After converting categoricals:\t{}".format(df.shape))
    

    # match test set and training set columns
    if enforce_cols is not None:
        to_drop = np.setdiff1d(df.columns, enforce_cols)
        to_add = np.setdiff1d(enforce_cols, df.columns)

        df.drop(to_drop, axis=1, inplace=True)
        df = df.assign(**{c: 0 for c in to_add})
    
    df.fillna(0, inplace=True)
    
    return df

In [20]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]

In [21]:
print("Country A")
aX_train =  pre_process_data(a_train.drop('poor', axis=1))
ay_train = np.ravel(a_train.poor)

print("\nCountry B")
bX_train =  pre_process_data(b_train.drop('poor', axis=1))
by_train = np.ravel(b_train.poor)

print("\nCountry C")
cX_train =  pre_process_data(c_train.drop('poor', axis=1))
cy_train = np.ravel(c_train.poor)

Country A
Input shape:	(8203, 344)
After standardization (8203, 344)
After converting categoricals:	(8203, 859)

Country B
Input shape:	(8203, 344)
After standardization (8203, 344)
After converting categoricals:	(8203, 859)

Country C
Input shape:	(8203, 344)
After standardization (8203, 344)
After converting categoricals:	(8203, 859)


In [22]:
# load test data
a_test = pd.read_csv(datadir+'A_hhold_test.csv', index_col='id')
b_test = pd.read_csv(datadir+'B_hhold_test.csv', index_col='id')
c_test = pd.read_csv(datadir+'C_hhold_test.csv', index_col='id')
# process the test data
a_test = pre_process_data(a_test, enforce_cols=aX_train.columns)
b_test = pre_process_data(b_test, enforce_cols=bX_train.columns)
c_test = pre_process_data(c_test, enforce_cols=cX_train.columns)

Input shape:	(4041, 344)
After standardization (4041, 344)
After converting categoricals:	(4041, 851)
Input shape:	(1604, 441)
After standardization (1604, 441)
After converting categoricals:	(1604, 1419)
Input shape:	(3187, 163)
After standardization (3187, 163)
After converting categoricals:	(3187, 773)


In [23]:
aX_train.head()

,nEsgxvAq,OMtioXZZ,YFMZwKrU,TiwRslOh,wBXbHZmp_DkQlr,wBXbHZmp_JhtDR,SlDKnCuu_GUusz,SlDKnCuu_alLXR,KAJOWiiw_BIZns,KAJOWiiw_TuovO,...,JCDeZBXq_UyAms,HGPWuGlV_WKNwg,HGPWuGlV_vkbkA,GDUPaBQs_qCEuA,GDUPaBQs_qQxrL,WuwrCsIY_AITFl,WuwrCsIY_GAZGl,AlDbXTlZ_aQeIm,AlDbXTlZ_cecIq,country_A
id,,,,,,,,,,,,,,,,,,,,,
46107,-16.0,21,-2.0,-7,0,1,1,0,0,1,...,0,0,1,0,1,1,0,1,0,1
82739,-10.0,12,-3.0,-1,0,1,1,0,0,1,...,1,0,1,0,1,1,0,0,1,1
9646,-4.0,12,-3.0,-7,0,1,1,0,1,0,...,1,0,1,0,1,1,0,0,1,1
10975,-4.0,3,-4.0,-1,0,1,1,0,0,1,...,1,0,1,0,1,1,0,0,1,1
16463,-4.0,21,-4.0,-5,0,1,0,1,0,1,...,1,0,1,0,1,0,1,1,0,1


In [24]:
a_test.head()

,nEsgxvAq,OMtioXZZ,YFMZwKrU,TiwRslOh,wBXbHZmp_DkQlr,wBXbHZmp_JhtDR,SlDKnCuu_GUusz,SlDKnCuu_alLXR,KAJOWiiw_BIZns,KAJOWiiw_TuovO,...,AlDbXTlZ_cecIq,country_A,LjvKYNON_tFrTi,UjuNwfjv_ACFXG,UjuNwfjv_ZBMyY,UjuNwfjv_ujWYB,nKoaotpH_HQYUq,nSzbETYS_yTPHV,tHFrzjai_MkATH,tHFrzjai_xBZqe
id,,,,,,,,,,,,,,,,,,,,,
418,-4.0,12,-3.0,-9,0,1,1,0,0,1,...,1,1,0,0,0,0,0,0,0,0
41249,-4.0,30,-2.0,-1,0,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
16205,-10.0,21,-2.0,-11,0,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
97501,-4.0,30,-2.0,3,0,1,0,1,1,0,...,1,1,0,0,0,0,0,0,0,0
67756,-16.0,30,-3.0,-7,0,1,1,0,1,0,...,1,1,0,0,0,0,0,0,0,0


In [25]:
def probabilities(alg,model_a,model_b,model_c, a_test, b_test, c_test):
    a_test=a_test[aX_train.columns]
    b_test=b_test[bX_train.columns]
    c_test=c_test[cX_train.columns]
    
    a_preds = model_a.predict_proba(a_test)
    b_preds = model_b.predict_proba(b_test)
    c_preds = model_c.predict_proba(c_test)
    
    # convert preds to data frames
    a_sub = make_country_sub(a_preds, a_test, 'A')
    b_sub = make_country_sub(b_preds, b_test, 'B')
    c_sub = make_country_sub(c_preds, c_test, 'C')
    
    filename='submission_'+str(alg)+'.csv'
    file= pd.concat([a_sub, b_sub, c_sub])
    file.to_csv( filename)
    
    return file.head()    

In [26]:
def fit_grid(grid_search):
    print('\rFitting... ', end='')
    grid_search.fit(aX_train,ay_train)
    model_a = grid_search.best_estimator_
    print(grid_search.best_params_)
    acc=model_a.score(aX_train,ay_train)
    print('A',acc)
    #print(model_a)
    
    print('\rFitting... ', end='')
    grid_search.fit(bX_train,by_train)
    model_b = grid_search.best_estimator_
    print(grid_search.best_params_)
    acc=model_b.score(bX_train,by_train)
    print('B',acc)
    #print(model_b)
    
    print('\rFitting... ', end='')
    grid_search.fit(cX_train,cy_train)
    model_c = grid_search.best_estimator_
    print(grid_search.best_params_)
    acc=model_c.score(cX_train,cy_train)
    print('C',acc)
    #print(model_c)
    
    return model_a, model_b, model_c

## Model parameters

In [27]:
aX_train.head()

,nEsgxvAq,OMtioXZZ,YFMZwKrU,TiwRslOh,wBXbHZmp_DkQlr,wBXbHZmp_JhtDR,SlDKnCuu_GUusz,SlDKnCuu_alLXR,KAJOWiiw_BIZns,KAJOWiiw_TuovO,...,JCDeZBXq_UyAms,HGPWuGlV_WKNwg,HGPWuGlV_vkbkA,GDUPaBQs_qCEuA,GDUPaBQs_qQxrL,WuwrCsIY_AITFl,WuwrCsIY_GAZGl,AlDbXTlZ_aQeIm,AlDbXTlZ_cecIq,country_A
id,,,,,,,,,,,,,,,,,,,,,
46107,-16.0,21,-2.0,-7,0,1,1,0,0,1,...,0,0,1,0,1,1,0,1,0,1
82739,-10.0,12,-3.0,-1,0,1,1,0,0,1,...,1,0,1,0,1,1,0,0,1,1
9646,-4.0,12,-3.0,-7,0,1,1,0,1,0,...,1,0,1,0,1,1,0,0,1,1
10975,-4.0,3,-4.0,-1,0,1,1,0,0,1,...,1,0,1,0,1,1,0,0,1,1
16463,-4.0,21,-4.0,-5,0,1,0,1,0,1,...,1,0,1,0,1,0,1,1,0,1


In [28]:
score='roc_auc'
jobs=-1
myfolds=StratifiedKFold(n_splits=4, shuffle=True)
#myfolds=3

## XGBOOST

In [29]:
# Define GridSearch parameters grid for XGBoost


parameter_grid_xgb = {
    #'booster':['gbtree','gblinear'],
     'reg_alpha':[1e-5, 1e-2, 0.1, 1],
     'max_depth': list(range(3,6,1)),
    #'gamma':[i/10.0 for i in range(3,5)],
     #'min_child_weight':list(range(4,6,2)),
    'n_estimators':list(range(50,600,100)),
    'learning_rate': [  0.09, 0.1, 1.1, 1.2] , 
     #'subsample':[i/10.0 for i in range(6,9)]
    #'colsample_bytree':[i/10.0 for i in range(7,9)]
}

xgb_grid_search = GridSearchCV(XGBClassifier(), parameter_grid_xgb, cv=myfolds, scoring=score, verbose=2, refit=True, n_jobs=jobs)

In [30]:
model_a, model_b, model_c=fit_grid(xgb_grid_search)

Fitting... Fitting 4 folds for each of 288 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 33.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 114.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 177.1min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 255.9min
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed: 297.0min finished


{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 250, 'reg_alpha': 1e-05}
A 0.948799219798
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Fitting... Fitting 4 folds for each of 288 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 26.6min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 117.0min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 218.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 297.6min
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed: 338.5min finished


{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 250, 'reg_alpha': 0.01}
B 0.948555406559
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Fitting... Fitting 4 folds for each of 288 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 24.8min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 72.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 134.8min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 219.7min
[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed: 298.3min finished


{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 250, 'reg_alpha': 0.1}
C 0.949408752895
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [31]:
probabilities('xgb',model_a,model_b,model_c, a_test, b_test, c_test)

,country,poor
id,,
418,A,0.892417
41249,A,0.000286
16205,A,0.806257
97501,A,0.000518
67756,A,0.972265


In [32]:
model_a, model_b, model_c

(XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
        n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
        reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=4, min_child_weight=1, missing=None, n_estimators=250,
        n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
        reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
        max_depth=4, min_child_weight=1, missing=None, n_es

In [33]:
stop here

SyntaxError: invalid syntax (<ipython-input-33-f9ef205a8f26>, line 1)

## AdaBoost

In [ ]:
parameter_grid_ada = {
    'n_estimators': list(range(50,1300,100)),
    'learning_rate': [ 0.001, 0.005, 0.009, 0.01, 0.1, 0.2, 0.3]   
}

ada_grid_search = GridSearchCV(AdaBoostClassifier(), parameter_grid_ada, cv=myfolds, scoring=score, verbose=2, refit=True, n_jobs=jobs)

In [ ]:
model_a, model_b, model_c=fit_grid(ada_grid_search)

In [ ]:
probabilities('ada',model_a,model_b,model_c, a_test, b_test, c_test)

In [ ]:
model_a

In [ ]:
model_b

In [ ]:
model_c

In [ ]:
stop here

## Gradient Boosting Classifier

In [ ]:
# Define GridSearch parameters grid for XGBoost
#https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/

parameter_grid_gbc = {
    'max_depth': list(range(3,9,2)),
    'n_estimators': list(range(50,500,50)),
    'learning_rate': [ 0.001, 0.01, 0.1, 0.2, 0.3, 0.4]
}

# Apply GridSearchCV

gbc_grid_search = GridSearchCV(GradientBoostingClassifier(), parameter_grid_gbc, 
                               cv=myfolds, scoring=score, verbose=2, refit=True, n_jobs=jobs)

In [ ]:
model_a, model_b, model_c=fit_grid(gbc_grid_search)

In [ ]:
 probabilities('gbc',model_a,model_b,model_c, a_test, b_test, c_test)

In [ ]:
ñkdxn

## Random Forest

In [ ]:
# Define GridSearch parameters grid for Random Forest
print('Defining parameter grid for Random Forest.')
param_grid_rf = {
    'min_samples_leaf': np.logspace(0, 5, num=5, base=4, endpoint=False, dtype=int),
    'min_samples_split': [ 10, 30, 40],
    'max_depth': list(range(3,8,2)),
    'n_estimators': list(range(20,550,60))
}

In [ ]:
# Apply GridSearchCV
print('Initialize GridSearchCV instance for RF.')
rf_grid_search = GridSearchCV(RandomForestClassifier(criterion='entropy'), param_grid_rf, cv=None, scoring='roc_auc', refit=True 
                              ,n_jobs=3,verbose=2)

In [ ]:
print('\rFitting RF... ', end='')
rf_grid_search.fit(aX_train, ay_train)
model_a = rf_grid_search.best_estimator_
print(model_a)

In [ ]:
print('\rFitting RF... ', end='')
rf_grid_search.fit(bX_train, by_train)
model_b = rf_grid_search.best_estimator_
print(model_b)

In [ ]:
print('\rFitting RF... ', end='')
rf_grid_search.fit(cX_train, cy_train)
model_c = rf_grid_search.best_estimator_
print(model_b)

In [ ]:
a_preds = model_a.predict_proba(a_test)
b_preds = model_b.predict_proba(b_test)
c_preds = model_c.predict_proba(c_test)

# convert preds to data frames
a_sub = make_country_sub(a_preds, a_test, 'A')
b_sub = make_country_sub(b_preds, b_test, 'B')
c_sub = make_country_sub(c_preds, c_test, 'C')
submission_rf = pd.concat([a_sub, b_sub, c_sub])
submission_rf.to_csv('submission_rf.csv')
submission_rf.head()

In [ ]:
'''
# RF Feature Importances
importances = model_a.feature_importances_
indices = np.argsort(importances)[::-1]

# Plot the feature importances
fig = plt.figure(figsize=(20,8))
plt.bar(range(len(indices)), importances[indices], color="r", align="center")
plt.title('Feature Importances Bar Plot')
plt.xlabel('Feature Name')
plt.ylabel('Feature Importance')
plt.xticks(range(len(indices)), train_val.drop('Y', axis=1).columns[indices], rotation=45, horizontalalignment='right')
plt.xlim([-1, len(indices)])
plt.title('Feature Importance Graph for RF')
plt.show()
print('\rSaving feature_importances barplot... ', end='')
fig.savefig(rf_dir + 'feature_importances_rf.png')
print('done.')
'''

In [ ]:
Stop

## Logistic Regression

In [ ]:
#1st, set up a paramater grid
param_grid_lr = {'C':[10**i for i in range(-4, 4)], 'penalty':['l1', 'l2']}

#2nd, call the GridSearchCV class, use LogisticRegression and 'roc_auc' for scoring
lr_grid_search = GridSearchCV(LogisticRegression(), param_grid_lr, cv = None, n_jobs=2, scoring = 'log_loss', refit=True) 


In [ ]:
lr_grid_search.fit(aX_train, ay_train)
model_a=lr_grid_search.best_estimator_

In [ ]:
lr_grid_search.fit(bX_train, by_train)
model_b=lr_grid_search.best_estimator_

In [ ]:
lr_grid_search.fit(cX_train, cy_train)
model_c=lr_grid_search.best_estimator_

In [ ]:
#Predict probabilities
a_preds = model_a.predict_proba(a_test)
b_preds = model_b.predict_proba(b_test)
c_preds = model_c.predict_proba(c_test)

In [ ]:
# convert preds to data frames
a_sub = make_country_sub(a_preds, a_test, 'A')
b_sub = make_country_sub(b_preds, b_test, 'B')
c_sub = make_country_sub(c_preds, c_test, 'C')
submission_lr = pd.concat([a_sub, b_sub, c_sub])
submission_lr.to_csv('submission_lr.csv')

In [ ]:
submission_lr.head()